In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import argparse
import pprint
import copy

import sys
os.environ["CUDA_VISIBLE_DEVICES"]='0'
sys.path.append("/data/ShenShuo/workspace/POPLIN")

from dotmap import DotMap

from dmbrl.misc.MBExp import MBExperiment
from dmbrl.controllers.MPC import MPC
from dmbrl.config import create_config
from dmbrl.misc import logger

import dmbrl.env
import gym
import tensorflow as tf
import numpy as np

/data/ShenShuo/.conda/envs/poplin/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/ShenShuo/.conda/envs/poplin/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/ShenShuo/.conda/envs/poplin/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/ShenShuo/.conda/envs/poplin/lib/python3.6/s

In [2]:
# def main(env, ctrl_type, ctrl_args, overrides, logdir, args):

# dO = (18,), dA = (6,)
env = 'gym_swimmer'
ctrl_type = 'MPC'
ctrl_args = [['model-type', 'DE'], ['prop-type', 'E'], ['opt-type', 'CEM']]
overrides = [['exp_cfg.exp_cfg.ntrain_iters', '50']]
logdir = '/data/ShenShuo/workspace/POPLIN/log/debug'
    

ctrl_args = DotMap(**{key: val for (key, val) in ctrl_args})
cfg = create_config(env, ctrl_type, ctrl_args, overrides, logdir)
logger.info('\n' + pprint.pformat(cfg))

# add the part of popsize
if ctrl_type == "MPC":
    cfg.exp_cfg.exp_cfg.policy = MPC(cfg.ctrl_cfg)

# cfg.exp_cfg.misc = copy.copy(cfg)
exp = MBExperiment(cfg.exp_cfg)

if not os.path.exists(exp.logdir):
    os.makedirs(exp.logdir)
with open(os.path.join(exp.logdir, "config.txt"), "w") as f:
    f.write(pprint.pformat(cfg.toDict()))

##################################debug cem cost#############################
from mbbl.env.gym_env import walker
env = walker.env(env_name=env, rand_seed=1234,
                            misc_info={'reset_type': 'gym'})
A = env.action_space.shape
print(A)

sess = exp.policy.model.sess
sess.run(tf.global_variables_initializer())

obs = env.reset()
exp.policy.sy_cur_obs.load(obs, sess)
cur_obs = sess.run(exp.policy.sy_cur_obs)
print(cur_obs)
ac_seqs = tf.random.uniform((500, 30 * 2), minval=-1, maxval=1, dtype=tf.float32)
costs, pred_trajs = exp.policy._compile_cost(ac_seqs, get_pred_trajs=True,
                        cem_type=None, tf_data_dict=None)
costs, pred_trajs, ac_seqs = sess.run([costs, pred_trajs, ac_seqs])   
#############################################################################

/data/ShenShuo/.conda/envs/poplin/lib/python3.6/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)



[0115 20:17:26 @<ipython-input-2-63189227a4ff>:13] 
{'ctrl_cfg': {'cem_cfg': {'cem_type': 'POPLINA-INIT',
                          'discriminator_act_type': 'leaky_relu',
                          'discriminator_ent_lambda': 0.001,
                          'discriminator_epochs': 40,
                          'discriminator_gradient_penalty_coeff': 10.0,
                          'discriminator_lr': 0.003,
                          'discriminator_minibatch_size': 64,
                          'discriminator_network_shape': [64, 64],
                          'discriminator_norm_type': None,
                          'gan_type': 'GAN',
                          'init_var': 0.25,
                          'minibatch_size': 64,
                          'pct_testset': 0.2,
                          'policy_epochs': 100,
                          'policy_lr': 0.003,
                          'policy_network_shape': [64, 64],
                          'policy_weight_decay': 1e-05,
      


[0115 20:17:27 @NN.py:77] Created an ensemble of 5 neural networks for critic without variance predictions.
[0115 20:17:27 @NN.py:77] Created an ensemble of 5 neural networks for critic without variance predictions.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[0115 20:17:28 @MPC.py:184] Created an MPC controller, prop mode E, 1 particles. Ignoring variance.
[0115 20:17:28 @MPC.py:184] Created an MPC controller, prop mode E, 1 particles. Ignoring variance.
[0115 20:17:28 @MPC.py:195] Trajectory prediction logging is disabled.
[0115 20:17:28 @MPC.py:195] Trajectory prediction logging is disabled.
[0115 20:17:28 @logger.py:98] Log file set to /data/ShenShuo/workspace/POPLIN/log/debug/2021-01-15--20:17:28/Friday_15_January_2021_08:17PM.log
[0115 20:17:28 @logger.py:98] Log file set to /data/ShenShuo/workspace/POPLIN/log/debug/2021-01-15--20:17:28/Friday_15_January_2021_08:17PM.log
[0115 20:17:28 @MBExp.py:77] Starting the experiments
[0115

In [3]:
def obs_cost_fn(obs):
    """ @brief:
            see mbbl.env.gym_env.walker.py for reward details
    """
    if isinstance(obs, np.ndarray):
        velocity_cost = -obs[:, 3]  # the qvel for the root-x joint
        return velocity_cost
    else:
        velocity_cost = -obs[:, 3]  # the qvel for the root-x joint
        return velocity_cost

def ac_cost_fn(acs):
    if isinstance(acs, np.ndarray):
        return 0.0001 * np.sum(np.square(acs), axis=1)
    else:
        return 0.0001 * tf.reduce_sum(tf.square(acs), axis=1)

print(costs.shape, pred_trajs.shape, ac_seqs.shape)

# reshape to cem format
# super critic 
# ac_seqs must transopse as in origin code

# this cannont work
# ac_seqs = ac_seqs.reshape(30, 500, 6)

# this works fine 
ac_seqs = np.reshape(ac_seqs, [500, 30, 2])
ac_seqs = np.reshape(np.tile(
        np.transpose(ac_seqs, [1, 0, 2])[:, :, None],
        [1, 1, 1, 1]
), [30, -1, 2])

pred_trajs = np.squeeze(pred_trajs)

(500,) (31, 500, 1, 8) (500, 60)


In [8]:
#################################
#             same              #
#################################

# this mean it is wrong
# POPLIN env should use cur_obs to calculate reward

cost = 0
for i in range(30):
    act = ac_seqs[i ,0,:]
    obs = pred_trajs[i + 1, 0, :]
    cost += (obs_cost_fn(obs.reshape(1, -1)) + ac_cost_fn(act.reshape(1, -1)))
cost_cem = costs[0]
print('\tcost:%f, \tcost_cem: %f, \tcost_diff:%f' % (cost, cost_cem, cost - cost_cem))

	cost:1.914499, 	cost_cem: 1.914499, 	cost_diff:0.000000
